In [6]:
import json
import random
import numpy as np
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder


In [8]:
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\manas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\manas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\manas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\manas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\manas\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       tagge

True

In [9]:
lemmatizer = WordNetLemmatizer()

# Load the intents JSON file
with open('intents.json') as file:
    intents = json.load(file)

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.']

# Tokenizing the words and creating classes
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        # Add documents in the corpus
        documents.append((word_list, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatizing and lowering each word, and removing duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))

classes = sorted(set(classes))

print(words)
print(classes)


["'s", ',', 'anyone', 'are', 'awesome', 'bye', 'chatting', 'day', 'for', 'good', 'goodbye', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'later', 'me', 'next', 'nice', 'see', 'simplilearn', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'you']
['goodbye', 'greeting', 'query', 'thanks']


In [11]:
# Create training data
training = []
output_empty = [0] * len(classes)

# Create a bag of words for each sentence
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle the features and turn it into separate lists
random.shuffle(training)

# Extract features and labels
train_x = []
train_y = []

for feature, label in training:
    train_x.append(feature)
    train_y.append(label)

# Convert lists to NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)


In [14]:
# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('chatbot_model.h5')
print("Model created and saved as 'chatbot_model.h5'")


Epoch 1/200
4/4 [==============================] - 1s 6ms/step - loss: 1.3798 - accuracy: 0.2222
Epoch 2/200
4/4 [==============================] - 0s 0s/step - loss: 1.3409 - accuracy: 0.2778
Epoch 3/200
4/4 [==============================] - 0s 401us/step - loss: 1.2279 - accuracy: 0.3889
Epoch 4/200
4/4 [==============================] - 0s 5ms/step - loss: 1.2313 - accuracy: 0.5556
Epoch 5/200
4/4 [==============================] - 0s 0s/step - loss: 1.2478 - accuracy: 0.4444
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 1.1571 - accuracy: 0.5000
Epoch 7/200
4/4 [==============================] - 0s 5ms/step - loss: 1.1528 - accuracy: 0.5556
Epoch 8/200
4/4 [==============================] - 0s 0s/step - loss: 1.1375 - accuracy: 0.5000
Epoch 9/200
4/4 [==============================] - 0s 3ms/step - loss: 0.9905 - accuracy: 0.5000
Epoch 10/200
4/4 [==============================] - 0s 5ms/step - loss: 0.9978 - accuracy: 0.6111
Epoch 11/200
4/4 [============

C:\Users\manas\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model created and saved as 'chatbot_model.h5'
